In [1]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from openmmtools import cache, utils
from perses.dispersed import feptasks
from perses.dispersed.feptasks import minimize
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState, RESTStateV2
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools
from simtk.openmm import app
cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())


Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1607619912205/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: CPU


INFO:rdkit:Enabling RDKit 2020.09.3 jupyter extensions


Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1607619912205/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: CPU


In [2]:
temperature = 298.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

In [3]:
# Create vanilla system for alanine dipeptide in solvent
# ala_solvent = AlanineDipeptideExplicit()
ala_solvent = AlanineDipeptideExplicit(use_dispersion_correction=True, nonbondedMethod=app.PME)
ala_solvent.system.removeForce(4)
# ala_solvent.system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
# ala_solvent.system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.CutoffPeriodic)
# ala_solvent.system.getForce(3).setUseDispersionCorrection(False)
# ala_solvent.system.getForce(3).setReactionFieldDielectric(78.3)
# ala_solvent.system.getForce(3).setEwaldErrorTolerance(5e-4)


In [4]:
# Zero nonzero 1,4 exceptions
for exception in range(ala_solvent.system.getForce(3).getNumExceptions()):
    p1, p2, q, sigma, epsilon = ala_solvent.system.getForce(3).getExceptionParameters(exception)
    if q != 0 or epsilon != 0:
        ala_solvent.system.getForce(3).setExceptionParameters(exception, p1, p2, q*0, sigma, epsilon*0)
        
# Zero nonbonded interactions
for particle in range(ala_solvent.system.getForce(3).getNumParticles()):
    q, sigma, epsilon = ala_solvent.system.getForce(3).getParticleParameters(particle)
    ala_solvent.system.getForce(3).setParticleParameters(particle, q*0, sigma, epsilon)
    

In [4]:
openmm.NonbondedForce.PME

4

In [5]:
ala_solvent.system.getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b80855cf2a0> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b80855cf1b0> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b80855cf1e0> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b80855cf2d0> >]

In [6]:
ala_solvent.system.getForce(3).getUseSwitchingFunction()

True

In [7]:
ala_solvent.system.getForce(3).getSwitchingDistance()

Quantity(value=0.8500000000000001, unit=nanometer)

In [8]:
ala_solvent.system.getForce(3).getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

In [5]:
# Set temperatures
T_min = 298.0 * unit.kelvin
T = 1200 * unit.kelvin
# T = 298.0 * unit.kelvin

# Set sys, pos, top
system = ala_solvent.system
positions = ala_solvent.positions
topology = ala_solvent.topology

In [6]:
# Compute energy for REST system
# Create REST system
res1 = list(topology.residues())[1]
rest_atoms = [atom.index for atom in res1.atoms()]
factory = RESTTopologyFactory(system, solute_region=rest_atoms, use_dispersion_correction=True)
# factory = REST2(system, solute_region=rest_atoms, use_dispersion_correction=False)
# factory = REST3(system, solute_region=rest_atoms, use_dispersion_correction=False)

REST_system = factory.REST_system

# Create thermodynamic state
lambda_zero_alchemical_state = RESTStateV2.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate,
                                                          composable_states=[lambda_zero_alchemical_state])

# Set alchemical parameters
beta_0 = 1 / (kB * T_min)
beta_m = 1 / (kB * T)
compound_thermodynamic_state.set_alchemical_parameters(beta_0, beta_m)

# Minimize and save energy
integrator = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context = compound_thermodynamic_state.create_context(integrator)
context.setPositions(positions)
sampler_state = SamplerState.from_context(context)
REST_energy = compound_thermodynamic_state.reduced_potential(sampler_state)

# Compute energy for non-RESTified system
# Determine regions and scaling factors
solute = rest_atoms
solvent = [i for i in range(topology.getNumAtoms()) if i not in rest_atoms]
solute_scaling = beta_m / beta_0
inter_scaling = np.sqrt(beta_m / beta_0)

# Scale the terms in the bond force appropriately
bond_force = system.getForce(0)
for bond in range(bond_force.getNumBonds()):
    p1, p2, length, k = bond_force.getBondParameters(bond)
    if p1 in solute and p2 in solute:
        bond_force.setBondParameters(bond, p1, p2, length, k * solute_scaling)
    elif (p1 in solute and p2 in solvent) or (p1 in solvent and p2 in solute):
        bond_force.setBondParameters(bond, p1, p2, length, k * inter_scaling)

# Scale the terms in the angle force appropriately
angle_force = system.getForce(1)
for angle_index in range(angle_force.getNumAngles()):
    p1, p2, p3, angle, k = angle_force.getAngleParameters(angle_index)
    if p1 in solute and p2 in solute and p3 in solute:
        angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k * solute_scaling)
    elif set([p1, p2, p3]).intersection(set(solute)) != set() and set([p1, p2, p3]).intersection(
            set(solvent)) != set():
        angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k * inter_scaling)

# Scale the terms in the torsion force appropriately
torsion_force = system.getForce(2)
for torsion_index in range(torsion_force.getNumTorsions()):
    p1, p2, p3, p4, periodicity, phase, k = torsion_force.getTorsionParameters(torsion_index)
    if p1 in solute and p2 in solute and p3 in solute and p4 in solute:
        torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k * solute_scaling)
    elif set([p1, p2, p3, p4]).intersection(set(solute)) != set() and set([p1, p2, p3, p4]).intersection(
            set(solvent)) != set():
        torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k * inter_scaling)

# Scale the exceptions in the nonbonded force appropriately
nb_force = system.getForce(3)
for nb_index in range(nb_force.getNumExceptions()):
    p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(nb_index)
    if p1 in solute and p2 in solute:
        nb_force.setExceptionParameters(nb_index, p1, p2, solute_scaling * chargeProd, sigma, solute_scaling * epsilon)
    elif (p1 in solute and p2 in solvent) or (p1 in solvent and p2 in solute):
        nb_force.setExceptionParameters(nb_index, p1, p2, inter_scaling * chargeProd, sigma, inter_scaling * epsilon)

# Scale nonbonded interactions for solute-solute region by adding exceptions for all pairs of atoms
exception_pairs = [tuple(sorted([nb_force.getExceptionParameters(nb_index)[0], nb_force.getExceptionParameters(nb_index)[1]])) for nb_index in range(nb_force.getNumExceptions())]
solute_pairs = set([tuple(sorted(pair)) for pair in list(itertools.product(solute, solute))])
for pair in list(solute_pairs):
    p1 = pair[0]
    p2 = pair[1]
    p1_charge, p1_sigma, p1_epsilon = nb_force.getParticleParameters(p1)
    p2_charge, p2_sigma, p2_epsilon = nb_force.getParticleParameters(p2)
    if p1 != p2:
        if pair not in exception_pairs:
            nb_force.addException(p1, p2, p1_charge * p2_charge * solute_scaling, 0.5 * (p1_sigma + p2_sigma),
                                  np.sqrt(p1_epsilon * p2_epsilon) * solute_scaling)

# Scale nonbonded interactions for inter region by adding exceptions for all pairs of atoms
for pair in list(itertools.product(solute, solvent)):
    p1 = pair[0]
    p2 = int(pair[1])  # otherwise, will be a numpy int
    p1_charge, p1_sigma, p1_epsilon = nb_force.getParticleParameters(p1)
    p2_charge, p2_sigma, p2_epsilon = nb_force.getParticleParameters(p2)
    if tuple(sorted(pair)) not in exception_pairs:
        nb_force.addException(p1, p2, p1_charge * p2_charge * inter_scaling, 0.5 * (p1_sigma + p2_sigma), np.sqrt(p1_epsilon * p2_epsilon) * inter_scaling)

# Get energy
thermostate = ThermodynamicState(system, temperature=T_min)
integrator = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context = thermostate.create_context(integrator)
context.setPositions(positions)
sampler_state = SamplerState.from_context(context)
nonREST_energy = thermostate.reduced_potential(sampler_state)

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]
INFO:REST:No unknown forces.


In [7]:
REST_energy

1214.697199938652

In [8]:
nonREST_energy

1213.911116955877

In [9]:
REST_energy - nonREST_energy

0.7860829827748148

In [9]:
compound_thermodynamic_state.electrostatic_scale

-0.2952541829378008

In [10]:
compound_thermodynamic_state.steric_scale

-0.5033333333333334

In [13]:
solute_scaling - 1

-0.5033333333333334

In [14]:
inter_scaling - 1

-0.2952541829378008

In [10]:

for exception in range(REST_system.getForce(3).getNumExceptions()):
    p1, p2, q, sigma, epsilon = REST_system.getForce(3).getExceptionParameters(exception)
    print(p1, p2, q, sigma, epsilon)
#     if q != 0 or epsilon != 0:
#         ala_solvent.system.getForce(3).setExceptionParameters(exception, p1, p2, q*0, sigma, epsilon*0)

5 7 -0.0 e**2 0.2014500181682605 nm 0.0 kJ/mol
4 9 0.0 e**2 0.29355112752934986 nm 0.0 kJ/mol
3 5 -0.0 e**2 0.28047273444524545 nm 0.0 kJ/mol
3 6 -0.0 e**2 0.2949765655645629 nm 0.0 kJ/mol
2 5 -0.0 e**2 0.28047273444524545 nm 0.0 kJ/mol
2 6 -0.0 e**2 0.2949765655645629 nm 0.0 kJ/mol
1 7 -0.0 e**2 0.22343739850856315 nm 0.0 kJ/mol
0 5 -0.0 e**2 0.28047273444524545 nm 0.0 kJ/mol
0 6 -0.0 e**2 0.2949765655645629 nm 0.0 kJ/mol
15 17 -0.0 e**2 0.2014500181682605 nm 0.0 kJ/mol
14 19 0.0 e**2 0.29355112752934986 nm 0.0 kJ/mol
14 20 0.0 e**2 0.29355112752934986 nm 0.0 kJ/mol
14 21 0.0 e**2 0.29355112752934986 nm 0.0 kJ/mol
13 14 0.0 e**2 0.3024601147602527 nm 0.0 kJ/mol
12 14 0.0 e**2 0.3024601147602527 nm 0.0 kJ/mol
11 14 0.0 e**2 0.3024601147602527 nm 0.0 kJ/mol
9 11 0.0 e**2 0.2560442914951194 nm 0.0 kJ/mol
9 12 0.0 e**2 0.2560442914951194 nm 0.0 kJ/mol
9 13 0.0 e**2 0.2560442914951194 nm 0.0 kJ/mol
9 15 -0.0 e**2 0.2715637472143426 nm 0.0 kJ/mol
9 16 -0.0 e**2 0.2860675783336601 nm 0.0 kJ/

503 504 0.0 e**2 0.1 nm 0.0 kJ/mol
505 506 0.0 e**2 0.1 nm 0.0 kJ/mol
505 507 0.0 e**2 0.1 nm 0.0 kJ/mol
506 507 0.0 e**2 0.1 nm 0.0 kJ/mol
508 509 0.0 e**2 0.1 nm 0.0 kJ/mol
508 510 0.0 e**2 0.1 nm 0.0 kJ/mol
509 510 0.0 e**2 0.1 nm 0.0 kJ/mol
511 512 0.0 e**2 0.1 nm 0.0 kJ/mol
511 513 0.0 e**2 0.1 nm 0.0 kJ/mol
512 513 0.0 e**2 0.1 nm 0.0 kJ/mol
514 515 0.0 e**2 0.1 nm 0.0 kJ/mol
514 516 0.0 e**2 0.1 nm 0.0 kJ/mol
515 516 0.0 e**2 0.1 nm 0.0 kJ/mol
517 518 0.0 e**2 0.1 nm 0.0 kJ/mol
517 519 0.0 e**2 0.1 nm 0.0 kJ/mol
518 519 0.0 e**2 0.1 nm 0.0 kJ/mol
520 521 0.0 e**2 0.1 nm 0.0 kJ/mol
520 522 0.0 e**2 0.1 nm 0.0 kJ/mol
521 522 0.0 e**2 0.1 nm 0.0 kJ/mol
523 524 0.0 e**2 0.1 nm 0.0 kJ/mol
523 525 0.0 e**2 0.1 nm 0.0 kJ/mol
524 525 0.0 e**2 0.1 nm 0.0 kJ/mol
526 527 0.0 e**2 0.1 nm 0.0 kJ/mol
526 528 0.0 e**2 0.1 nm 0.0 kJ/mol
527 528 0.0 e**2 0.1 nm 0.0 kJ/mol
529 530 0.0 e**2 0.1 nm 0.0 kJ/mol
529 531 0.0 e**2 0.1 nm 0.0 kJ/mol
530 531 0.0 e**2 0.1 nm 0.0 kJ/mol
532 533 0.0 e**2 0.1

1103 1104 0.0 e**2 0.1 nm 0.0 kJ/mol
1105 1106 0.0 e**2 0.1 nm 0.0 kJ/mol
1105 1107 0.0 e**2 0.1 nm 0.0 kJ/mol
1106 1107 0.0 e**2 0.1 nm 0.0 kJ/mol
1108 1109 0.0 e**2 0.1 nm 0.0 kJ/mol
1108 1110 0.0 e**2 0.1 nm 0.0 kJ/mol
1109 1110 0.0 e**2 0.1 nm 0.0 kJ/mol
1111 1112 0.0 e**2 0.1 nm 0.0 kJ/mol
1111 1113 0.0 e**2 0.1 nm 0.0 kJ/mol
1112 1113 0.0 e**2 0.1 nm 0.0 kJ/mol
1114 1115 0.0 e**2 0.1 nm 0.0 kJ/mol
1114 1116 0.0 e**2 0.1 nm 0.0 kJ/mol
1115 1116 0.0 e**2 0.1 nm 0.0 kJ/mol
1117 1118 0.0 e**2 0.1 nm 0.0 kJ/mol
1117 1119 0.0 e**2 0.1 nm 0.0 kJ/mol
1118 1119 0.0 e**2 0.1 nm 0.0 kJ/mol
1120 1121 0.0 e**2 0.1 nm 0.0 kJ/mol
1120 1122 0.0 e**2 0.1 nm 0.0 kJ/mol
1121 1122 0.0 e**2 0.1 nm 0.0 kJ/mol
1123 1124 0.0 e**2 0.1 nm 0.0 kJ/mol
1123 1125 0.0 e**2 0.1 nm 0.0 kJ/mol
1124 1125 0.0 e**2 0.1 nm 0.0 kJ/mol
1126 1127 0.0 e**2 0.1 nm 0.0 kJ/mol
1126 1128 0.0 e**2 0.1 nm 0.0 kJ/mol
1127 1128 0.0 e**2 0.1 nm 0.0 kJ/mol
1129 1130 0.0 e**2 0.1 nm 0.0 kJ/mol
1129 1131 0.0 e**2 0.1 nm 0.0 kJ/mol
1

1703 1704 0.0 e**2 0.1 nm 0.0 kJ/mol
1705 1706 0.0 e**2 0.1 nm 0.0 kJ/mol
1705 1707 0.0 e**2 0.1 nm 0.0 kJ/mol
1706 1707 0.0 e**2 0.1 nm 0.0 kJ/mol
1708 1709 0.0 e**2 0.1 nm 0.0 kJ/mol
1708 1710 0.0 e**2 0.1 nm 0.0 kJ/mol
1709 1710 0.0 e**2 0.1 nm 0.0 kJ/mol
1711 1712 0.0 e**2 0.1 nm 0.0 kJ/mol
1711 1713 0.0 e**2 0.1 nm 0.0 kJ/mol
1712 1713 0.0 e**2 0.1 nm 0.0 kJ/mol
1714 1715 0.0 e**2 0.1 nm 0.0 kJ/mol
1714 1716 0.0 e**2 0.1 nm 0.0 kJ/mol
1715 1716 0.0 e**2 0.1 nm 0.0 kJ/mol
1717 1718 0.0 e**2 0.1 nm 0.0 kJ/mol
1717 1719 0.0 e**2 0.1 nm 0.0 kJ/mol
1718 1719 0.0 e**2 0.1 nm 0.0 kJ/mol
1720 1721 0.0 e**2 0.1 nm 0.0 kJ/mol
1720 1722 0.0 e**2 0.1 nm 0.0 kJ/mol
1721 1722 0.0 e**2 0.1 nm 0.0 kJ/mol
1723 1724 0.0 e**2 0.1 nm 0.0 kJ/mol
1723 1725 0.0 e**2 0.1 nm 0.0 kJ/mol
1724 1725 0.0 e**2 0.1 nm 0.0 kJ/mol
1726 1727 0.0 e**2 0.1 nm 0.0 kJ/mol
1726 1728 0.0 e**2 0.1 nm 0.0 kJ/mol
1727 1728 0.0 e**2 0.1 nm 0.0 kJ/mol
1729 1730 0.0 e**2 0.1 nm 0.0 kJ/mol
1729 1731 0.0 e**2 0.1 nm 0.0 kJ/mol
1